**I. Синтаксис**

Синтаксическую разметку продуктивнее всего получать с помощью библиотек **stanza** или **spacy** (spacy вы можете рассмотреть самостоятельно: https://spacy.io/usage). В них много разных моделей для различных языков и достаточно понятное извлечение тэгов. Продолжим работу с stanza, c которой мы уже знакомы.

Если эта библиотека ещё не установлена, установите: 

In [ ]:
! pip install stanza

In [ ]:
import stanza
stanza.download("ru")
nlp_stanza = stanza.Pipeline(lang="ru", processors="tokenize, pos, lemma, depparse, ner")

In [ ]:
text = 'Компания Газпром любит красивых кошек.' 

Разметим наше предложение и посмотрим на результат:

In [ ]:
doc = nlp_stanza(text)
print(doc)

Из этого мы можем аккуратно собрать всю морфологическую и синтаксическую информацию. В stanza мы работаем сначала с каждым предложением индивидуально:

In [ ]:
for sentence in doc.sentences:
    print(sentence)

А затем со словами: 

In [ ]:
for sentence in doc.sentences:
    for word in sentence.words:
        print(word)

In [ ]:
doc.ents

Отсюда уже можно по ключу получит нужную информацию:

In [ ]:
for sentence in doc.sentences:
    for word in sentence.words:
        print(word.id, word.text, word.upos, word.deprel, word.head)

Если, например, наша задача заключается в том, чтобы отследить какое количество разных типов подлежащих использует пишущий текст автор, мы должны использовать условные конструкции.

In [ ]:
text = 'Она любит зеленые кактусы. Он не любит зеленых камней. Газпром приносит счастье в дом. Он отчаянаная голова'
doc = nlp_stanza(text)
counter_pron = 0
counter_noun = 0
for sentence in doc.sentences:
    for word in sentence.words:
        if word.upos == 'PRON' and word.deprel == 'nsubj':
            counter_pron += 1
        elif word.upos == 'NOUN' and word.deprel == 'nsubj':
            counter_noun += 1
print(counter_pron)
print(counter_noun)

Теперь склеим это в удобочитаемую табличку: 

In [ ]:
import pandas as pd

In [ ]:
text = 'Я люблю красивых кошек.' 
doc = nlp_stanza(text)

In [ ]:
list_of_rows = [[word.id, word.text, word.upos, word.deprel, word.head, sentence.words[word.head-1].text] for sentence in doc.sentences for word in sentence.words]
#это краткая запись цикла выше: не пугайтесь!

In [ ]:
list_of_rows

In [ ]:
df_sentence = pd.DataFrame(list_of_rows, columns=['id', 'token', 'pos', 'synt_tag', 'head_id', 'head_tok'])
df_sentence

А дальнейшая работа с подсчетом статистик - дело техники!  

In [ ]:
df_sentence.loc[df_sentence['synt_tag'] == 'nsubj']

Список возможных тэгов, указывающих на тип зависимости здесь: https://universaldependencies.org/u/dep/

Столбец head обозначает, какое слово является вершиной конкретного слова. Например, в предложении выше алгоритм считает, что вершиной предложения является "люблю" (root), вершиной "кошек" - слово "люблю". Т.о., у нас в этом коротком предложении есть несколько групп:  
**группа глагола** (люблю -> я, люблю -> кошек, люблю -> .) и  
**группа существительного** (кошек -> красивых).

Чуть-чуть изменим предложение выше.

In [ ]:
text_two = 'Юный чтец любит древние книги, которые находит в библиотеке.'
doc = nlp_stanza(text_two)
for sentence in doc.sentences:
    print(sentence)

In [ ]:
list_of_rows = [[word.id, word.text, word.upos, word.deprel, word.head, sentence.words[word.head-1].text] for sentence in doc.sentences for word in sentence.words]
df_sentence = pd.DataFrame(list_of_rows, columns=['id', 'token', 'pos', 'synt_tag', 'head_id', 'head_tok'])
df_sentence

Глазами мы видим следующие группы:  
1. группа глагола главного предложения: любит -> чтец (**nsubj**, отношение сказуемого и подлежащего), любит -> книги (**obj**, отношение дополнения), любит -> . (**punct**, отношение пунктуации).
2. группа подлежащего главного предложения: чтец -> юный (**amod**, отношение **adjectival modifier**, определение).
3. группа прямого дополнения глагола главного предложения: книги -> древние (**amod**, определение), книги -> находит (**acl:relcl**, здесь сложнее. acl значит clausal modifier of noun, приименная клауза, которая при этом имеет надстройку в виде relcl - relative clause, относительное придаточное; алгоритм строит дерево не от существительного к относительному союзу, а к вершине зависимого предложения, но в самом отношении между существительным и зависимой вершиной предложения модифицируется для указания на тип придаточного предложения).
4. группа глагола зависимого предложения: находит -> которые (nsubj, наше относительное местоимение является подлежащим зависимой клаузы), находит -> библиотеке (**obl**, oblique - т.е. зависимое от глагола существительное в косвенном падеже, не являющееся прямым дополнением), находит ->  , (**punct**).
5. группа зависимого существительного в придаточном: библиотеке -> в (**case** нужен для указаний отношений существительного с предлогом, когда предлог требует какого-то косвенного падежда).

**Задание 1.** Выберите собственное предложение, оберните его в размеченную табличку и посмотрите, какие группы есть в этом предложении, разберите его так же, как в тексте выше.

На самом деле мы хотели бы собирать все группы не вручную, а автоматически.

In [ ]:
text = 'Юный чтец любит древние книги, которые находит в библиотеке.'
doc = nlp_stanza(text)
list_of_rows = [[word.id, word.text, word.upos, word.deprel, word.head, sentence.words[word.head-1].text] for sentence in doc.sentences for word in sentence.words]
list_of_rows

Не самое эффективное, но достаточно действенное решение проблемы со сбором групп:

In [ ]:
text = 'Юный чтец любит древние книги, которые находит в библиотеке.'
doc = nlp_stanza(text)
list_of_rows = [[word.id, word.text, word.upos, word.deprel, word.head, sentence.words[word.head-1].text] for word in sentence.words for sentence in doc.sentences]
df_sentence = pd.DataFrame(list_of_rows, columns=['id', 'token', 'pos', 'synt_tag', 'head_id', 'head_tok'])

all_deps = [] #cюда мы сохраним пары вершина+зависимые
for token in list_of_rows: #будем итерироваться по токенам
    #print(token[5])
    rslt_dep = df_sentence.loc[df_sentence['head_tok'] == token[5]] #нас интересует только 5-й элемент списка, это токен вершины
    #print(rslt_dep)   
    dep = list(rslt_dep['token'].values) #получим значение столбца
    if (token[5], dep, len(dep)) not in all_deps: #если ещё нет такой вершины с зависимыми
        all_deps.append((token[5], dep, len(dep))) #сохраним их в all_deps
print(all_deps)

Посчитаем среднее количество зависимых в каждой из групп:

In [ ]:
counter = 0
for element in all_deps:
    counter += element[2]
avg_group = counter / len(all_deps)
avg_group

In [ ]:
text_1 = 'Екатерининская эпоха ознаменовалась максимальным закрепощением крестьян и всесторонним расширением привилегий дворянства.'
doc = nlp_stanza(text_1)
list_of_rows = [[word.id, word.text, word.upos, word.deprel, word.head, sentence.words[word.head-1].text] for sentence in doc.sentences for word in sentence.words]
df_sentence = pd.DataFrame(list_of_rows, columns=['id', 'token', 'pos', 'synt_tag', 'head_id', 'head_tok'])

all_deps_1 = [] #cюда мы сохраним пары вершина+зависимые
for token in list_of_rows: #будем итерироваться по токенам
    #print(token[5])
    rslt_dep = df_sentence.loc[df_sentence['head_tok'] == token[5]] #нас интересует только 5-й элемент списка, это токен вершины
    #print(rslt_dep)   
    dep = list(rslt_dep['token'].values) #получим значение столбца
    if (token[5], dep, len(dep)) not in all_deps: #если ещё нет такой вершины с зависимыми
        all_deps_1.append((token[5], dep, len(dep))) #сохраним их в all_deps

text_2 = 'У меня большая семья из шести человек: я, мама, папа, старшая сестра, бабушка и дедушка.'
doc = nlp_stanza(text_2)
list_of_rows = [[word.id, word.text, word.upos, word.deprel, word.head, sentence.words[word.head-1].text] for sentence in doc.sentences for word in sentence.words]
df_sentence = pd.DataFrame(list_of_rows, columns=['id', 'token', 'pos', 'synt_tag', 'head_id', 'head_tok'])
all_deps_2 = [] #cюда мы сохраним пары вершина+зависимые
for token in list_of_rows: #будем итерироваться по токенам
    #print(token[5])
    rslt_dep = df_sentence.loc[df_sentence['head_tok'] == token[5]] #нас интересует только 5-й элемент списка, это токен вершины
    #print(rslt_dep)   
    dep = list(rslt_dep['token'].values) #получим значение столбца
    if (token[5], dep, len(dep)) not in all_deps: #если ещё нет такой вершины с зависимыми
        all_deps_2.append((token[5], dep, len(dep))) #сохраним их в all_deps

In [ ]:
counter_1 = 0
for element in all_deps_1:
    counter_1 += element[2]
avg_group_1 = counter_1 / len(all_deps_1)
avg_group_1

In [ ]:
counter_2 = 0
for element in all_deps_2:
    counter_2 += element[2]
avg_group_2 = counter_2 / len(all_deps_2)
avg_group_2

In [ ]:
print('Количество групп в нашем предложении:', len(all_deps)-1) #вычитаем единицу, потому отношение root лишнее

А если у нас несколько предложений в тексте?

In [ ]:
text = 'Юный чтец любит древние книги, которые находит в библиотеке. Этот чтец является большим экспертом.'
doc = nlp_stanza(text)
list_of_rows = [[word.id, word.text, word.upos, word.deprel, word.head, sentence.words[word.head-1].text] for sentence in doc.sentences for word in sentence.words]
df_sentence = pd.DataFrame(list_of_rows, columns=['id', 'token', 'pos', 'synt_tag', 'head_id', 'head_tok'])
df_sentence

Будет ошибка. Это потому что stanza разбирает текст по предложениям. Поправим.

In [ ]:
list_of_rows = []
counter = 0
for sentence in doc.sentences:
    counter += 1  
    for word in sentence.words:
       # print([word.id, word.text, word.upos, word.deprel, word.head, sentence.words[word.head-1].text])
        list_of_rows.append([counter, word.id, word.text, word.upos, word.deprel, word.head, sentence.words[word.head-1].text])

df_sentence = pd.DataFrame(list_of_rows, columns=['sent_id', 'id', 'token', 'pos', 'synt_tag', 'head_id', 'head_tok'])
df_sentence

Если мы оставим предыдущий код для подсчета зависимых, то он будет ошибаться:

In [ ]:
all_deps = [] #cюда мы сохраним пары вершина+зависимые
for token in list_of_rows: #будем итерироваться по токенам
   # print(token[6])
    rslt_dep = df_sentence.loc[df_sentence['head_tok'] == token[6]] #нас интересует только 5-й элемент списка, это токен вершины
    dep = list(rslt_dep['token'].values) #получим значение столбца
    if (token[6], dep) not in all_deps: #если ещё нет такой вершины с зависимыми
        all_deps.append((token[6], dep)) #сохраним их в all_deps
print(all_deps)

Нужно ввести контроль, чтобы мы работали внутри предложения.

In [ ]:
all_deps = [] #cюда мы сохраним пары вершина+зависимые
for token in list_of_rows: #будем итерироваться по токенам
   # print(token[6])
    rslt_dep = df_sentence.loc[(df_sentence['head_tok'] == token[6]) & (df_sentence['sent_id'] == token[0])]  #нас интересует только 5-й элемент списка, это токен вершины
    dep = list(rslt_dep['token'].values) #получим значение столбца
    if (token[6], dep, len(dep)) not in all_deps: #если ещё нет такой вершины с зависимыми
        all_deps.append((token[6], dep, len(dep))) #сохраним их в all_deps
print(all_deps)

Ура! Для полноты картины в принципе можно делить тексты по предложениям:

In [ ]:
all_deps = [] #cюда мы сохраним пары вершина+зависимые
for token in list_of_rows: #будем итерироваться по токенам
   # print(token[6])
    rslt_dep = df_sentence.loc[(df_sentence['head_tok'] == token[6]) & (df_sentence['sent_id'] == token[0])]  #нас интересует только 5-й элемент списка, это токен вершины
    dep = list(rslt_dep['token'].values) #получим значение столбца
    if (token[0], token[6], dep, len(dep)) not in all_deps: #если ещё нет такой вершины с зависимыми
        all_deps.append((token[0], token[6], dep, len(dep))) #сохраним их в all_deps

for deps in all_deps:
    print(deps)

Количество групп в предложении можно назвать **"шириной"** синтаксического дерева. Таким образом, в первом предложении текста ширина равна 5 (минус root, как мы помним), во втором - 3.

**Задание 2**. Разметьте небольшой текст. Сколько в среднем зависимых у каждого из слов? Сколько в принципе групп в каждом предложении? Какое среднее значение групп по тексту?

In [ ]:
text_1 = 'Дочь князя Ангальт-Цербстского, Екатерина взошла на престол в результате дворцового переворота против своего мужа — Петра III, вскоре погибшего при невыясненных обстоятельствах (возможно, он был убит)[3]. Она взошла на престол, следуя прецеденту, созданному Екатериной I, сменившей своего умершего мужа Петра Великого в 1725 году. Екатерининская эпоха ознаменовалась максимальным закрепощением крестьян и всесторонним расширением привилегий дворянства. При Екатерине Великой границы Российской империи были значительно сдвинуты на запад (разделы Речи Посполитой) и на юг (присоединение Новороссии, Крыма, отчасти Кавказа). Были созданы условия для свободной деятельности всех конфессий; положение староверов (раскольников) было облегчено. Система государственного управления при Екатерине Второй впервые со времени Петра I была реформирована. Сенат был разделён на шесть департаментов, возглавляемых обер-прокурорами, возглавил Сенат генерал-прокурор. Общие полномочия Сената были сокращены: в частности, он лишился законодательной инициативы. Была проведена губернская реформа, в ходе которой было преобразовано провинциальное управление в наместничествах. Расходы на содержание чиновничьего аппарата резко возросли. Характерной особенностью правления Екатерины II стал фаворитизм, государственные расходы на фаворитов исчислялись десятками миллионов рублей. Повсеместными были коррупция и злоупотребления чиновников. На фоне происходившей в ряде других стран промышленной революции, в России использовался в основном ручной труд без развития механизации и применения новых технологий, поскольку Екатерина II считала, что машины наносят вред государству, сокращая численность работающих. В структуре экспорта совсем не было готовых изделий, только сырьё и полуфабрикаты, а 80—90 % импорта составляли зарубежные промышленные изделия. К концу правления Екатерины II Россия находилась в тяжёлом экономическом кризисе при полном крушении финансовой системы, общая сумма долгов правительства составляла 205 млн рублей. Внешние займы Екатерины II и начисленные на них проценты были полностью погашены только в 1891 году. У Екатерины II на всём протяжении её правления были десятки любовников, некоторые из которых оказывали большое влияние на внутреннюю и внешнюю политику. Распутство императрицы проявлялось в откровенно вызывающей форме и способствовало падению нравов дворянства. Екатерина II увлекалась литературной деятельностью, собирала шедевры живописи, состояла в переписке с французскими просветителями. Императрица предприняла ряд попыток преобразований в духе просвещённого абсолютизма, но эти преобразования имели ограниченный характер.'
text_2 = 'Привет! Меня зовут Андрей! Мне 12 лет. У меня много друзей. Моего лучшего друга зовут Паша. Мы одного возраста и учимся в одном классе. Он живет в соседнем доме. Мы ходим в школу пешком вместе. У него короткие светлые волосы, зеленые глаза и добрая улыбка. Он очень худой и высокий. Паша выше меня на целую голову. Он часто смеется и готов помочь. Мы обычно играем в футбол с другими ребятами после школы. Мы всегда играем в одной команде. Паша очень любит спорт и мечтает стать тренером, когда вырастет. Иногда мы идем ко мне домой после футбола и делаем вместе домашнюю работу. Паша всегда объясняет мне сложные задачи, потому что учится лучше меня, особенно по математике.'

Итак, у нас есть размеченный корпус или текст. Например, что мы можем извлекать с помощью синтаксиса? Например, что именно синтаксически сочетается с конкретным словом. Вернемся к нашей табличке, но разметим что-нибудь другое:

In [ ]:
with open('Joshua.txt', encoding='utf-8') as txt:
    text = txt.read()
    doc = nlp_stanza(text[:50000])
    list_of_rows = [[word.id, word.text, word.lemma, word.upos, word.deprel, word.head, sentence.words[word.head-1].text, sentence.words[word.head-1].lemma] for sentence in doc.sentences for word in sentence.words]
    df_joshua = pd.DataFrame(list_of_rows, columns=['id', 'token', 'lemma','pos', 'synt_tag', 'head_id', 'head_tok', 'head_lemma'])

In [ ]:
df_joshua

Давайте найдем теперь все адъективные модификаторы (amod) cлова "море". Сначала посмотрим просто на amod:

In [ ]:
df_joshua.loc[df_joshua['synt_tag'] == 'amod']

Пока это только искомые слова. Но мы не зря вывели лемму вершины:

In [ ]:
df_joshua.loc[(df_joshua['synt_tag'] == 'amod') & (df_joshua['head_lemma'] == 'море')]

Теперь осталось посчитать, сколько раз каждый из модификаторов повторился:

In [ ]:
df_joshua.loc[(df_joshua['synt_tag'] == 'amod') & (df_joshua['head_lemma'] == 'море')].value_counts()

Увы, тут проверка на уникальность, а у нас как минимум меняются словоформы и вершины. Поправим так, чтобы учитывалась только лемма:

In [ ]:
df_amods = df_joshua.loc[(df_joshua['synt_tag'] == 'amod') & (df_joshua['head_lemma'] == 'море')]
df_amods

In [ ]:
df_amods.groupby('lemma').size().reset_index(name='counts')